# Decorators

## Material

http://www.artima.com/weblogs/viewpost.jsp?thread=240808

http://simeonfranklin.com/blog/2012/jul/1/python-decorators-in-12-steps/

http://thecodeship.com/patterns/guide-to-python-function-decorators/

## Basic facts

Decorators are basically syntax sugar, which allows encapsulating a function (or an object) withing another function or object which returns it. For example:

In [15]:
from __future__ import print_function, division

def outer(some_func):
    def inner():
        print("before some_func")
        ret = some_func() # 1
        return ret + 1
    return inner

def foo():
    return 1

decorated = outer(foo) # 2
decorated()

before some_func


2

the last 4 lines can be written as:

In [16]:
@outer
def foo2():
    return 1

foo2()


before some_func


2

## Class decorators

Definition of a `class` decorator is pretty straightforward: the function is passed to `__init__`, and the arguments to `__call__`:

In [33]:
class entryExit(object):

    def __init__(self, f):
        print("inside __init__")
        self.f = f

    def __call__(self, *args):
        print("inside __call__")
        print("Entering", self.f.__name__)
        self.f(*args)
        print("Exited", self.f.__name__)

@entryExit
def func1(x):
    print("inside func1()")
    print(x)

@entryExit
def func2(x):
    print("inside func2()")
    print(x)

#func1(1)
#func2(2)

inside __init__
inside __init__


you see that `__init__` is called at the decoration step, while `__call__` is called every time you call the function:



In [34]:
func1(1)
func2(2)
func1(3)

inside __call__
Entering func1
inside func1()
1
Exited func1
inside __call__
Entering func2
inside func2()
2
Exited func2
inside __call__
Entering func1
inside func1()
3
Exited func1


NB: you won't be automatically maintain the function's name!

In [19]:
a = func1
print(dir(a))
print(a.f.__name__)


['__call__', '__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'f']
func1


Using a decorator with arguments on a `class` is less straightforward... in this case, the arguments of the decorator are passed to `__init__`, while the function to `__call__`, that must wrap and return it:

In [30]:
class Wrapper(object):

    def __init__(self, a):
        print("inside __init__")
        self.a = a

    def __call__(self, f):
        print("inside __call__")
        def wrapped_f(*args):
            self.f = f
            print("Entering", self.f.__name__)
            self.f(*args)
            print(self.a)
            
            print("Exited", self.f.__name__)
        return wrapped_f
    
@Wrapper(1)
def fool(x):
    print("I'm a fool ", x)
    
fool("test")


inside __init__
inside __call__
Entering fool
I'm a fool  test
1
Exited fool


You can also pass an object as Decorator argument

In [26]:
class Wrapper(object):

    def __init__(self, a):
        print("inside __init__")
        self.a = a

    def __call__(self, f):
        print("inside __call__")
        def wrapped_f(*args):
            self.f = f
            print("Entering", self.f.__name__)
            self.f(*args)
            if isinstance(self.a, object):
                print("It's an object, containing:")
                for k in dir(self.a):
                    if k[0] != "_":
                        print("   " + k)
            print("Exited", self.f.__name__)
        return wrapped_f

class State(object):
    d = {}
    
state = State()


@Wrapper(state)
def fool(x):
    print("inside I'm a fool", x)
    
fool("test")


inside __init__
inside __call__
Entering fool
inside I'm a fool test
It's an object, containing:
   d
Exited fool


It has to be noted that in this case (`class` decorator with argument) both `__call__` and `__init__` are called only once when you decorate a `class`. So, if we call again `fool()`:

In [27]:
fool("test2")

Entering fool
inside I'm a fool test2
It's an object, containing:
   d
Exited fool
